<a href="https://colab.research.google.com/github/drscghosh/Testing/blob/master/DialGrifM3Model4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Sample data generation: Griffing Method-4 (F1 only, no reciprocals)
def generate_sample_data():
    parents = ['P1', 'P2', 'P3', 'P4']
    data = []
    np.random.seed(42)

    for i in range(len(parents)):
        for j in range(i + 1, len(parents)):
            cross = f"{parents[i]}x{parents[j]}"
            value = np.random.normal(loc=50, scale=5, size=3)  # 3 replications
            for rep, v in enumerate(value, 1):
                data.append([parents[i], parents[j], cross, rep, v])

    return pd.DataFrame(data, columns=['Parent1', 'Parent2', 'Cross', 'Rep', 'Yield'])

# Perform Griffing's Method-4 Analysis
def diallel_analysis_method4(df):
    print("\n=== Sample Dataset ===")
    print(df.head())

    # Model: Yield ~ Cross (fixed effect model)
    model = ols('Yield ~ C(Cross)', data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)

    print("\n=== ANOVA Table ===")
    print(anova_table)

    # GCA and SCA effects estimation
    crosses = df['Cross'].unique()
    parents = sorted(set(df['Parent1']) | set(df['Parent2']))
    n = len(parents)

    # Compute means
    cross_means = df.groupby('Cross')['Yield'].mean()
    grand_mean = df['Yield'].mean()

    # Create matrix for SCA and GCA
    gca = {p: 0.0 for p in parents}
    sca = {}

    # GCA estimation
    gca_counts = {p: 0 for p in parents}
    for cross in crosses:
        p1, p2 = cross.split('x')
        mean_yield = cross_means[cross]
        gca[p1] += mean_yield
        gca[p2] += mean_yield
        gca_counts[p1] += 1
        gca_counts[p2] += 1

    for p in parents:
        gca[p] = (gca[p] / gca_counts[p]) - grand_mean

    # SCA estimation
    for cross in crosses:
        p1, p2 = cross.split('x')
        sca_val = cross_means[cross] - (grand_mean + gca[p1] + gca[p2])
        sca[cross] = sca_val

    print("\n=== GCA Effects ===")
    for p in gca:
        print(f"{p}: {gca[p]:.3f}")

    print("\n=== SCA Effects ===")
    for c in sca:
        print(f"{c}: {sca[c]:.3f}")

# Run the analysis
if __name__ == "__main__":
    df = generate_sample_data()
    diallel_analysis_method4(df)



=== Sample Dataset ===
  Parent1 Parent2  Cross  Rep      Yield
0      P1      P2  P1xP2    1  52.483571
1      P1      P2  P1xP2    2  49.308678
2      P1      P2  P1xP2    3  53.238443
3      P1      P3  P1xP3    1  57.615149
4      P1      P3  P1xP3    2  48.829233

=== ANOVA Table ===
              sum_sq    df         F    PR(>F)
C(Cross)  155.987431   5.0  1.669099  0.216333
Residual  224.294508  12.0       NaN       NaN

=== GCA Effects ===
P1: 2.495
P2: -1.235
P3: -0.022
P4: -1.237

=== SCA Effects ===
P1xP2: 0.725
P1xP3: -0.408
P1xP4: 2.178
P2xP3: 0.920
P2xP4: -2.881
P3xP4: -0.535
